# Structure Optimization at Different Pressures

## Overview

This notebook demonstrates how to optimize the crystal structure of CO₂ at different pressures and phases using our custom energy functions and optimization routines.

### Learning Objectives
- Understand how to build CO₂ crystal structures
- Learn to set up pressure-dependent optimization
- Analyze structural changes with pressure
- Visualize optimization results

### Prerequisites
- Basic understanding of crystal structures
- Familiarity with Python and NumPy
- Understanding of pressure effects on solids

## 1. Setup and Imports

In [19]:
import numpy as np
import pandas as pd
import sys
import json
import pickle
from pathlib import Path
from datetime import datetime
import warnings
#warnings.filterwarnings('ignore')

# Crystal structure and optimization
from pymatgen.core import Structure, Lattice
from pymatgen.io.cif import CifWriter
from pymatgen.io.xyz import XYZ
from scipy.optimize import minimize
import matplotlib.pyplot as plt

# Custom CO₂ modules
from spacegroup import Pa3, Cmce, P42mnm, R3c
from symmetry import build_unit_cell
from energy import compute_energy_from_cell
from co2_potential import p1b, p2b, sapt

print("✓ All modules imported successfully")

✓ Successfully imported custom modules
Setup complete!


## 2. Data Storage System Setup

In [ ]:
class CO2OptimizationStorage:
    """
    Comprehensive storage system for CO₂ optimization results.
    """
    
    def __init__(self, base_dir="optimization_results"):
        self.base_dir = Path(base_dir)
        self.setup_directories()
        
    def setup_directories(self):
        """Create directory structure for organized storage."""
        self.base_dir.mkdir(exist_ok=True)
        (self.base_dir / "structures").mkdir(exist_ok=True)
        (self.base_dir / "raw_data").mkdir(exist_ok=True)
        (self.base_dir / "analysis").mkdir(exist_ok=True)
        
        print(f"✓ Storage directories created in: {self.base_dir.absolute()}")
    
    def extract_pef_parameters(self):
        """Extract parameters from potential energy functions."""
        try:
            # Try to extract parameters from the potential functions
            # This may need adjustment based on your actual function implementations
            pef_params = {
                'p1b_info': {
                    'function': 'p1b',
                    'description': 'Monomer potential',
                    'extracted_at': datetime.now().isoformat()
                },
                'p2b_info': {
                    'function': 'p2b', 
                    'description': 'Two-body potential',
                    'extracted_at': datetime.now().isoformat()
                },
                'sapt_info': {
                    'function': 'sapt',
                    'description': 'SAPT intermolecular potential',
                    'extracted_at': datetime.now().isoformat()
                }
            }
            
            # Try to get function signatures or docstrings
            for func_name, func in [('p1b', p1b), ('p2b', p2b), ('sapt', sapt)]:
                try:
                    pef_params[f'{func_name}_info']['docstring'] = func.__doc__
                    pef_params[f'{func_name}_info']['module'] = func.__module__
                except:
                    pass
                    
            return pef_params
            
        except Exception as e:
            print(f"Warning: Could not extract PEF parameters: {e}")
            return {"error": "Could not extract PEF parameters", "timestamp": datetime.now().isoformat()}
    
    def save_metadata(self):
        """Save metadata about the calculations."""
        metadata = {
            'creation_date': datetime.now().isoformat(),
            'software_versions': {
                'python': f"{sys.version}",
                'numpy': np.__version__,
                'pandas': pd.__version__,
            },
            'pef_parameters': self.extract_pef_parameters(),
            'units': {
                'pressure': 'GPa',
                'energy': 'kcal/mol', 
                'length': 'Angstrom',
                'angles': 'degrees'
            },
            'conversion_factors': {
                'gpa_angstrom3_to_kcal_per_mol': 0.1439
            }
        }
        
        with open(self.base_dir / "metadata.json", 'w') as f:
            json.dump(metadata, f, indent=2)
        
        print("✓ Metadata saved")
        return metadata
    
    def save_optimization_result(self, space_group, pressure, optimization_result, 
                                structure, convergence_info):
        """
        Save individual optimization result.
        
        Parameters:
        -----------
        space_group : str
            Space group name (Pa3, Cmce, etc.)
        pressure : float
            Pressure in GPa
        optimization_result : dict
            Results from scipy.optimize.minimize
        structure : pymatgen.Structure
            Optimized crystal structure
        convergence_info : dict
            Additional convergence information
        """
        
        # Create result dictionary
        result_data = {
            'calculation_info': {
                'space_group': space_group,
                'pressure_gpa': pressure,
                'timestamp': datetime.now().isoformat(),
                'calculation_type': 'pressure_dependent_optimization'
            },
            'optimized_parameters': self._extract_structure_parameters(structure, space_group, optimization_result),
            'energies': self._calculate_energy_breakdown(structure, convergence_info),
            'optimization_convergence': {
                'success': optimization_result.get('success', False),
                'iterations': optimization_result.get('nit', None),
                'function_evaluations': optimization_result.get('nfev', None),
                'final_gradient_norm': float(np.linalg.norm(optimization_result.get('jac', [0]))) if optimization_result.get('jac') is not None else None,
                'convergence_message': optimization_result.get('message', ''),
                'final_function_value': float(optimization_result.get('fun', np.inf)),
                **convergence_info
            },
            'structure_info': {
                'volume_angstrom3': structure.volume,
                'density_g_per_cm3': structure.density,
                'num_atoms': len(structure),
                'formula': structure.composition.reduced_formula
            }
        }
        
        # Save individual result file
        filename = f"{space_group.lower()}_{pressure:.1f}_gpa_result.json"
        filepath = self.base_dir / "raw_data" / filename
        
        with open(filepath, 'w') as f:
            json.dump(result_data, f, indent=2)
        
        # Save structure filesp
        self._save_structure_files(structure, space_group, pressure)
        
        print(f"✓ Saved optimization result: {filename}")
        return result_data
    
    def _extract_structure_parameters(self, structure, space_group, result):
        """Extract relevant parameters based on space group."""
        lattice = structure.lattice
        
        base_params = {
            'lattice_a': lattice.a,
            'lattice_b': lattice.b, 
            'lattice_c': lattice.c,
            'alpha': lattice.alpha,
            'beta': lattice.beta,
            'gamma': lattice.gamma
        }
        
        # Add space-group specific parameters
        # This would need to be enhanced based on your specific parameter definitions
        if space_group == 'Pa3':
            base_params['bond_length'] = result.get('x', {})[1]
        elif space_group == 'Cmce':
            base_params['bond_length'] = result.get('x', {})[3]
            base_params['bond_angle'] = result.get('x', {})[4]
        elif space_group == 'P42mnm':
            base_params['bond_length'] = result.get('x', {})[2]
        elif space_group == 'R3c':
            base_params['bond_length1'] = result.get('x', {})[2]
            base_params['bond_length2'] = result.get('x', {})[3]
            base_params['bond_angle_phi'] = result.get('x', {})[4]
            base_params['bond_angle_theta'] = result.get('x', {})[5]

        return base_params
    
    def _calculate_energy_breakdown(self, structure, convergence_info):
        """Calculate detailed energy breakdown."""
        try:
            total_energy = convergence_info.get("final_energy", None)
            total_enthalpy = convergence_info.get("final_enthalpy", None)
            
            return {
                'total_energy_kcal_per_mol': total_energy,
                'total_enthalpy_kcal_per_mol': total_enthalpy,
                'energy_per_molecule': total_energy / (len(structure) / 3),  # Assuming CO2 = 3 atoms
                'enthalpy_per_molecule': total_enthalpy / (len(structure) / 3),  # Assuming CO2 = 3 atoms
                'energy_density_kcal_per_mol_per_angstrom3': total_energy / structure.volume,
                'enthalpy_density_kcal_per_mol_per_angstrom3': total_enthalpy / structure.volume
            }
        except Exception as e:
            print(f"Warning: Could not calculate energy breakdown: {e}")
            return {
                'total_energy_kcal_per_mol': None,
                'total_enthalpy_kcal_per_mol': None,
                'energy_per_molecule': None,
                'enthalpy_per_molecule': None,
                'energy_density_kcal_per_mol_per_angstrom3': None,
                'enthalpy_density_kcal_per_mol_per_angstrom3': None,
                'error': str(e)
            }
    
    def _save_structure_files(self, structure, space_group, pressure):
        """Save structure in CIF and XYZ formats."""
        base_filename = f"{space_group.lower()}_{pressure:.1f}_gpa"
        
        # Save CIF file
        cif_writer = CifWriter(structure)
        cif_path = self.base_dir / "structures" / f"{base_filename}.cif"
        cif_writer.write_file(str(cif_path))
        
        # Save XYZ file (extended format with lattice info)
        try:
            # Convert to ASE
            from ase import Atoms
            positions = structure.cart_coords
            symbols = [str(site.specie) for site in structure]
            cell = structure.lattice.matrix
            atoms = Atoms(symbols=symbols, positions=positions, cell=cell, pbc=True)
        
            # Write XYZ
            xyz_path = self.base_dir / "structures" / f"{base_filename}.xyz"
            atoms.write(str(xyz_path), format='extxyz')
        except:
            print(f"Warning: Could not save XYZ file for {base_filename}")
    
    def update_master_index(self):
        """Update master index of all calculations."""
        # Scan all result files and create comprehensive index
        result_files = list((self.base_dir / "raw_data").glob("*_result.json"))
        
        index_data = {
            'last_updated': datetime.now().isoformat(),
            'total_calculations': len(result_files),
            'calculations': []
        }
        
        for result_file in result_files:
            try:
                with open(result_file, 'r') as f:
                    data = json.load(f)
                
                index_entry = {
                    'filename': result_file.name,
                    'space_group': data['calculation_info']['space_group'],
                    'pressure_gpa': data['calculation_info']['pressure_gpa'],
                    'success': data['optimization_convergence']['success'],
                    'total_energy': data['energies']['total_energy_kcal_per_mol'],
                    'timestamp': data['calculation_info']['timestamp']
                }
                index_data['calculations'].append(index_entry)
                
            except Exception as e:
                print(f"Warning: Could not index {result_file.name}: {e}")
        
        # Save master index
        with open(self.base_dir / "master_index.json", 'w') as f:
            json.dump(index_data, f, indent=2)
        
        print(f"✓ Master index updated with {len(index_data['calculations'])} calculations")
        return index_data

# Initialize storage system
storage = CO2OptimizationStorage()
metadata = storage.save_metadata()

## 3. Complete Pa3 Optimization 

In [ ]:
def optimize_pa3_at_pressure(pressure_gpa, initial_params=None, bounds=None, save_results=True):
    """
    Optimize Pa3 structure at given pressure with comprehensive data storage.
    
    Parameters:
    -----------
    pressure_gpa : float
        Pressure in GPa
    initial_params : dict
        Initial parameters {'a': value, 'bond_length': value}
    bounds : list
        Optimization bounds [(a_min, a_max), (bond_min, bond_max)]
    save_results : bool
        Whether to save results to storage system
        
    Returns:
    --------
    dict : Optimization results and structure
    """
    
    # Default parameters
    if initial_params is None:
        initial_params = {'a': 5.5, 'bond_length': 1.16}
    
    if bounds is None:
        bounds = [(4.0, 8.0), (1.0, 1.4)]
    
    # Conversion factor for PV term (GPa·Å³ to kcal/mol)
    pv_conversion = 0.1439
    
    def objective_with_pressure(x):
        """Objective function including pressure term."""
        a, bond_length = x
        
        try:
            # Create structure
            pa3_obj = Pa3(a=a, bond_length=bond_length)
            structure = build_unit_cell(pa3_obj)
            
            # Calculate energy
            energy = compute_energy_from_cell(structure, None)
            
            # Add PV term for Enthalpy
            pv_term = pressure_gpa * structure.volume * pv_conversion
            enthalpy = energy + pv_term
            
            return enthalpy
            
        except Exception as e:
            print(f"Error in objective function: {e}")
            return 1e10
    
    # Initial guess and energy
    x0 = [initial_params['a'], initial_params['bond_length']]

    # Create optimized structure
    pa3_obj = Pa3(a=x0[0], bond_length=x0[1])
    structure_int = build_unit_cell(pa3_obj)

    # Calculate initial energies
    initial_energy = compute_energy_from_cell(structure_int, None)
    initial_pv = pressure_gpa * structure_int.volume * pv_conversion
    initial_enthalpy = initial_energy + initial_pv

    #initial_energy = objective_with_pressure(x0)
    
    print(f"Optimizing Pa3 at {pressure_gpa:.1f} GPa")
    print(f"Initial parameters: a={x0[0]:.3f} Å, bond={x0[1]:.3f} Å")
    print(f"Initial enthalpy energy: {initial_enthalpy:.6f} kcal/mol")
    
    # Perform optimization
    result = minimize(
        objective_with_pressure, 
        x0, 
        method='L-BFGS-B', 
        bounds=bounds,
        options={
            'ftol': 1e-8,
            'gtol': 1e-6, 
            'maxiter': 200,
            'disp': False
        }
    )
    
    # Extract optimized parameters
    a_opt, bond_opt = result.x
    
    # Create optimized structure
    pa3_opt = Pa3(a=a_opt, bond_length=bond_opt)
    structure_opt = build_unit_cell(pa3_opt)
    
    # Calculate final energies
    final_energy = compute_energy_from_cell(structure_opt, None)
    final_pv = pressure_gpa * structure_opt.volume * pv_conversion
    final_enthalpy = final_energy + final_pv
    
    # Convergence information
    convergence_info = {
        'initial_parameters': {'a': x0[0], 'bond_length': x0[1]},
        'initial_energy': initial_energy,
        'initial_pv_term': initial_pv,
        'initial_enthalpy': initial_enthalpy,
        'final_energy': final_energy,
        'final_pv_term': final_pv,
        'final_enthalpy': final_enthalpy,
        'energy_improvement': initial_enthalpy - final_enthalpy,
        'pressure_gpa': pressure_gpa,
        'pv_conversion_factor': pv_conversion
    }
    
    print(f"Optimization completed!")
    print(f"Final parameters: a={a_opt:.4f} Å, bond={bond_opt:.4f} Å")
    print(f"Final enthalpy: {final_enthalpy:.6f} kcal/mol")
    print(f"Energy improvement: {convergence_info['energy_improvement']:.6f} kcal/mol")
    
    # Save results if requested
    if save_results:
        storage.save_optimization_result(
            space_group='Pa3',
            pressure=pressure_gpa,
            optimization_result=result,
            structure=structure_opt,
            convergence_info=convergence_info
        )
    
    return {
        'optimization_result': result,
        'optimized_structure': structure_opt,
        'optimized_parameters': {'a': a_opt, 'bond_length': bond_opt},
        'convergence_info': convergence_info,
        'success': result.success
    }


In [ ]:
# Example: Optimize Pa3 at a few pressure points
print("Running Pa3 optimizations at different pressures...")

test_pressures = [0.0, 5.0, 10.0]

pa3_results = {}

for pressure in test_pressures:
    try:
        result = optimize_pa3_at_pressure(pressure)
        pa3_results[pressure] = result
        print(f"✓ Completed optimization at {pressure} GPa")
    except Exception as e:
        print(f"✗ Failed optimization at {pressure} GPa: {e}")
        pa3_results[pressure] = None
    print()

# Update master index
storage.update_master_index()

### Pa3 Optimizations from 0 - 30 GPa at 0.2 GPa increments

Now we'll systematically optimize the Pa3 structure across a comprehensive pressure range to map out its stability and structural evolution under compression.

Use `np.arange()` to create the pressure list, for example: `pressures = np.arange(0, 30.1, 0.2)` for 0 to 30 GPa at 0.2 GPa increments.

In [ ]:
pressures = np.arange(0, 30.1, 0.2)
pa3_results = {}

for pressure in pressures:
    try:
        result = optimize_pa3_at_pressure(pressure)
        pa3_results[pressure] = result
        print(f"✓ Completed optimization at {pressure} GPa")
    except Exception as e:
        print(f"✗ Failed optimization at {pressure} GPa: {e}")
        pa3_results[pressure] = None
    print()

## 4. Complete Cmce Optimization

In [ ]:
def optimize_cmce_at_pressure(pressure_gpa, initial_params=None, bounds=None, save_results=True):
    """
    Optimize Cmce structure at given pressure.
    
    Parameters:
    -----------
    pressure_gpa : float
        Pressure in GPa
    initial_params : dict
        Initial parameters {'a': val, 'b': val, 'c': val, 'bond_length': val, 'bond_angle': val}
    bounds : list
        Optimization bounds for [a, b, c, bond_length, bond_angle]
    save_results : bool
        Whether to save results
        
    Returns:
    --------
    dict : Optimization results
    """
    
    # TODO: Set default parameters for Cmce
    if initial_params is None:
        initial_params = {
            'a': ___,           # Fill in reasonable initial value
            'b': ___,           # Fill in reasonable initial value  
            'c': ___,           # Fill in reasonable initial value
            'bond_length': ___,  # Fill in reasonable initial value
            'bond_angle': ___    # Fill in reasonable initial value
        }
    
    # TODO: Set reasonable bounds for Cmce optimization
    if bounds is None:
        bounds = [
            (___), # bounds for a
            (___), # bounds for b  
            (___), # bounds for c
            (___), # bounds for bond_length
            (___)  # bounds for bond_angle
        ]
    
    pv_conversion = 0.1439
    
    def objective_with_pressure(x):
        """Objective function for Cmce."""
        # TODO: Unpack optimization variables
        a, b, c, bond_length, bond_angle = x
        
        try:
            # TODO: Create Cmce structure
            cmce_obj = Cmce(a=___, b=___, c=___)
            
            # TODO: Adjust fractional coordinates
            cmce_obj.adjust_fractional_coords(___, ___)
            
            # TODO: Build unit cell
            structure = ___
            
            # TODO: Calculate energy (follow Pa3 example)
            energy = ___
            
            # TODO: Add PV term
            pv_term = ___
            gibbs_energy = ___
            
            return gibbs_energy
            
        except Exception as e:
            print(f"Error in Cmce objective: {e}")
            return 1e10
    
    # TODO: Set up initial guess from initial_params
    x0 = [___, ___, ___, ___, ___]
    
    print(f"TODO: Complete Cmce optimization at {pressure_gpa:.1f} GPa")
    
    # TODO: Implement the rest following Pa3 example
    # - Run minimize()
    # - Extract results
    # - Create final structure
    # - Calculate convergence info
    # - Save results if requested
    
    # Placeholder return
    return {
        'optimization_result': None,
        'optimized_structure': None,
        'optimized_parameters': None,
        'convergence_info': None,
        'success': False,
        'todo': 'Complete this function'
    }

### Cmce Optimizations from 0 - 30 GPa at 0.2 GPa increments

Now we'll systematically optimize the Cmce structure across a comprehensive pressure range to map out its stability and structural evolution under compression.

Use `np.arange()` to create the pressure list, for example: `pressures = np.arange(0, 30.1, 0.2)` for 0 to 30 GPa at 0.2 GPa increments.

In [ ]:
pressures = ____

for pressure in pressures:
    ____

## 5. Complete P42_mnm Optimization

In [ ]:
def optimize_p42mnm_at_pressure(pressure_gpa, initial_params=None, bounds=None, save_results=True):
    """
    Optimize P42mnm structure at given pressure.
    
    TODO for students: Complete this function
    """
    
    # TODO: Set default parameters for P42mnm (tetragonal: a, c, bond_length)
    if initial_params is None:
        initial_params = {
            'a': ___,           # Tetragonal a parameter
            'c': ___,           # Tetragonal c parameter  
            'bond_length': ___  # CO2 bond length
        }
    
    # TODO: Implement P42mnm optimization
    print(f"TODO: Complete P42mnm optimization at {pressure_gpa:.1f} GPa")
    
    return {
        'todo': 'Complete P42mnm optimization function'
    }

### P42mnm Optimizations from 0 - 30 GPa at 0.2 GPa increments

Now we'll systematically optimize the P42mnm structure across a comprehensive pressure range to map out its stability and structural evolution under compression.

Use `np.arange()` to create the pressure list, for example: `pressures = np.arange(0, 30.1, 0.2)` for 0 to 30 GPa at 0.2 GPa increments.

In [ ]:
pressures = ____

for pressure in pressures:
    ____

## 6. Complete R3c Optimization

In [ ]:
def optimize_r3c_at_pressure(pressure_gpa, initial_params=None, bounds=None, save_results=True):
    """
    Optimize R3c structure at given pressure.
    
    TODO for students: Complete this function (most challenging!)
    """
    
    # TODO: Set default parameters for R3c (hexagonal with multiple bond parameters)
    if initial_params is None:
        initial_params = {
            'a': ___,               # Hexagonal a parameter
            'c': ___,               # Hexagonal c parameter
            'bond_length1': ___,    # First bond length
            'bond_length2': ___,    # Second bond length  
            'bond_angle_phi': ___,  # Phi angle
            'bond_angle_theta': ___ # Theta angle
        }
    
    # TODO: Implement R3c optimization (most complex space group)
    print(f"TODO: Complete R3c optimization at {pressure_gpa:.1f} GPa")
    
    return {
        'todo': 'Complete R3c optimization function (advanced!)'
    }

### R3c Optimizations from 0 - 30 GPa at 0.2 GPa increments

Now we'll systematically optimize the Cmce structure across a comprehensive pressure range to map out its stability and structural evolution under compression.

Use `np.arange()` to create the pressure list, for example: `pressures = np.arange(0, 30.1, 0.2)` for 0 to 30 GPa at 0.2 GPa increments.

In [ ]:
pressures = ____

for pressure in pressures:
    ____

## 7. Analysis and Visualization

Let's analyze how the crystal structure changes with pressure and create informative plots.